# Import libaries and data

In [3]:
#Data Source
#https://www.kaggle.com/ahsen1330/us-police-shootings?select=shootings.csv
#
#
#
#
#
#Deploy Dash in Jupyter Notebook
#https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e


import pandas as pd

#################---For Geo-coding----###########################
#https://www.youtube.com/watch?v=ckPEY2KppHc
import requests
from urllib.parse import urlencode
#################################################################

df = pd.read_csv('shootings.csv')
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day

df['location'] = df['city'] + ', ' + df['state']

df.head(100)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,year,month,day,location
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,2015,1,2,"Shelton, WA"
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,2015,1,2,"Aloha, OR"
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,2015,1,3,"Wichita, KS"
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,2015,1,4,"San Francisco, CA"
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,2015,1,4,"Evans, CO"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,134,Joseph Paffen,2015-02-08,shot,gun,46.0,M,White,Orange County,FL,False,attack,Not fleeing,False,Guns,2015,2,8,"Orange County, FL"
96,135,Larry Hostetter,2015-02-08,shot,gun,41.0,M,White,Nocona,TX,False,attack,Not fleeing,False,Guns,2015,2,8,"Nocona, TX"
97,136,Kenneth Kreyssig,2015-02-09,shot,gun,61.0,M,White,Smyrna,ME,True,undetermined,Not fleeing,False,Guns,2015,2,9,"Smyrna, ME"
98,137,Desmond Luster,2015-02-09,shot,gun,45.0,M,Black,Dallas,TX,False,attack,Not fleeing,False,Guns,2015,2,9,"Dallas, TX"


# Creating the Class and Functions for geocoding API

In [ ]:
#Set up the credentials and authenticated Google Geocoding API and Places API
api_key = "API_code_here"

#I used most of the information from the instructional to get to the desired outcome
#https://www.youtube.com/watch?v=ckPEY2KppHc

class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None

    #This is a function that checks for an API key
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key is required")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()

#This function extracts the coordinates based on city and state, which will be based on "location" from the table above

def extract_lat_lng(city_and_state, data_type = 'json'):
    endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": city_and_state, "key": api_key}
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    r = requests.get(url)
    if r.status_code not in range(200, 299): 
        return {}
    latlng = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"), latlng.get("lng")



# Applying the extract_lat_lng function to dataframe

In [555]:
#Apply the function to each location in the dataframe shootings.csv to get the coordinates.
#Create a new column for coordinates
df['coordinates'] = df['location'].apply(extract_lat_lng)
df.head(100)



,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,year,month,day,location,coordinates
4521,5344,Derrick A. Elseth,2020-01-01,shot,gun,24.0,M,White,Richmond County,VA,False,other,Not fleeing,False,Guns,2020,1,1,"Richmond County, VA","(37.9175609, -76.7105632)"
4522,5403,Gerardo Antonio Conchas-Bustas,2020-01-01,shot,knife,20.0,M,Other,Denver,CO,False,attack,Not fleeing,True,Sharp objects,2020,1,1,"Denver, CO","(39.7392358, -104.990251)"
4523,5342,Gabriel Strickland,2020-01-01,shot and Tasered,toy weapon,25.0,M,White,Grass Valley,CA,True,attack,Not fleeing,False,Other unusual objects,2020,1,1,"Grass Valley, CA","(39.2190608, -121.0610606)"
4524,5339,Jeffery Dale Millsap,2020-01-02,shot,gun,26.0,M,White,Holt,MO,False,attack,Car,False,Guns,2020,1,2,"Holt, MO","(39.4527802, -94.3421687)"
4525,5349,Jamari Daiwon Tarver,2020-01-02,shot,vehicle,26.0,M,Black,North Las Vegas,NV,False,attack,Car,True,Vehicles,2020,1,2,"North Las Vegas, NV","(36.1988592, -115.1175013)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4616,5514,Juston Root,2020-02-07,shot,gun,41.0,M,White,Brookline,MA,True,attack,Car,False,Guns,2020,2,7,"Brookline, MA","(42.33176419999999, -71.1211635)"
4617,5524,Michael Collins,2020-02-07,shot,knife,38.0,M,White,Sapulpa,OK,True,other,Not fleeing,False,Sharp objects,2020,2,7,"Sapulpa, OK","(35.9987007, -96.1141664)"
4618,5518,Douglas Harold Hart,2020-02-08,shot,gun,67.0,M,White,Vestavia Hills,AL,False,attack,Not fleeing,False,Guns,2020,2,8,"Vestavia Hills, AL","(33.4487183, -86.7877668)"
4619,5517,Gordon Whitaker,2020-02-09,shot,gun,45.0,M,White,Kennewick,WA,False,attack,Foot,False,Guns,2020,2,9,"Kennewick, WA","(46.2086683, -119.119948)"


# Do final cleaning to Dataset

In [ ]:
#Do final cleaning to Dataset
coordinates = df['coordinates'].astype(str)

#Split the coordinates columns to seperate lat and long columns
df['lat'], df['lon'] = coordinates.str.split(', ').str

#remove the brackets from lat and long column
df['lat'] = df['lat'].str.replace('(','')
df['lon'] = df['lon'].str.replace(')','')

#send final data to CSV
df.to_csv('US_shootings_with_coordinates_2020.txt')